# 1. Installs, Imports and Settings

In [5]:
#!pip install spacy==3.1.1 #restart runtime after this
#!python -m spacy download en_core_web_sm

import csv
import pandas as pd
pd.options.display.max_rows
pd.set_option('display.max_colwidth', None)

from nltk.corpus import stopwords

import spacy
from spacy import displacy
from collections import Counter
import pandas as pd
pd.set_option("max_rows", 400)
pd.set_option("max_colwidth", 400)

import string
import operator
from itertools import islice
from collections import Counter
nlp = spacy.load('es_core_news_lg')

#  2. Reading files 

In [6]:
df = pd.read_csv('data/alertas.csv', sep="|")
df = df[df["Text"].notnull()]
print("Size of Data:", len(df))
print()
print("Columns : ", list(df.columns))

Size of Data: 666

Columns :  ['Filename', 'Text', 'Type', 'Year']


In [7]:
import re
exceptions = [
    "SAT \n \nSede Central Calle 55 Nº 10 32 Of 115 \nTels 3147300 Ext 2437 Telefax 6915300 \nEmail", 
    "DE LA", 
    "NI OTRO", 
    "Tels 3147300 Ext 2437 Telefax 6915300", 
    "Ext 2437 Telefax 6915300", 
    "Tels 3147300 Ext 2437 Telefax 6915300", 
    "Sede Central Calle 55 No", 
    "Tels 3147300 Ext 2437 Telefax 6915300", 
    "SAT Sede Central Calle 55 Nº 10 32 Of 115 Email", 
    "SAT Sede Central Calle 55 Nº 1032 Of 115 Email", 
    "SA Email",
    "SAT INFORME DE RIESGO No",
    "DEL CONFLICTO ARMADO Sistema",
    "LA VIDA LA LIBERTAD Y",
    "DELEGADA PARA LA EVALUACIÓN DEL RIESGO POBLACIÓN CIVIL",
    "SAT Sede Central Calle 55 Nº 1032 Of 115 Tels 3147300 Ext 2437 Telefax 6915300 Email", 
    "SAT Sede Central Calle 55 Nº 10 32 Of 115 Tels 3147300 Ext 2437 Telefax 6915300 Email", 
    "INFORME DE RIESGO No", 
    "LA INTEGRIDAD FISICA DE LA POBLACIÓN", 
    "Tels 3147300 Ext 2437 Telefax", 
    "DELEGADA PARA LA EVALUACIÓN", 
    "LA POBLACIÓN CIVIL", 
    "Tels 3147300 Ext 2437 Telefax 6915300 Correo Electrónico", 
    "VILLEGAS Defensor Delegado",
    "Tels 3147300 Ext 2452 Telefax",
    "2437 Telefax",
    "CHACÓN Defensor Delegado", 
    "Tels 3147300 ext 24372464 Fax ext 2452 Bogotá",
    "Tels 3147300 Ext 2437 Telefax ext", 
    "Tels 3147300 Ext 2452 Telefax", 
    "Telefax Ext 2452 Correo Electrónico", 
    "2437 Telefax", 
    "Tels 3147300 ext 24372464 Fax ext 2452 Bogotá", 
    "2464 Fax ext 2452 Bogotá",
    "X", 
    "Inminencia alta Urgente Grado 2 Urgencia intermedia Grado 3 No", 
    "SAT Sede Central Calle 55 Nº 1032 Bloque C Tercer piso Tels 3147300 Ext 2437 Telefax", 
    "O SITUACIÓN CRÓNICA",
    "AUTORIDADES VINCULADAS AL", 
    
    
]
def cleantext1(text):
    text = text.lower()
    text = text.replace("\n", " ")
    for s in exceptions:
        text = text.replace(s.lower(), " ")
    text = re.sub(' +', ' ', text)
    return text    

df["Text"] = df["Text"].apply(cleantext1)

# 3. Functions

In [11]:
stopw = stopwords.words('spanish') + ["at", "ala", "026", "1", "27", "06", "18", "www", "01", 
                                           "8000914814", "i", "t", "s", "wwwdefensoriagovco", "0", 
    "sede central", 
    "nº", 
    "calle 55", 
    "1032",
    "10", "32", "2", "email", "defensatyahoocom", "115", "3", "4", "", "5", "6", "7", "8", "9", "11", "19" ]

#m is the number of top ngrams.
def getNPartsOfSpeech(text, m, tag):
    words = []
    nlp.max_length = len(text) + 100
    document = nlp(text)
    for token in document:
        if token.pos_ == tag and token.lemma_ not in stopw:
            words.append(token.lemma_)
    w_tally = Counter(words) 
    df = pd.DataFrame(w_tally.most_common(), columns=['w', 'count'])
    
    return df[:m]

def printNPOS(data, m):
    postags = []
    
    verbs = getNPartsOfSpeech(data, m, 'VERB')
    #verbs += [(None, None)] * (m - len(verbs))
    
    adjs = getNPartsOfSpeech(data, m , 'ADJ')
    #adjs +=[(None, None)] * (m - len(adjs))
    
    nouns = getNPartsOfSpeech(data, m , 'NOUN')
    #nouns +=[(None, None)] * (m - len(nouns))
    
    result = pd.concat([verbs, adjs, nouns], axis=1, join="inner")
    result.columns = ["Verbs", "Abs Freq", "Adjectives", "Abs Freq", "Nouns", "Abs Freq"]
    return result

In [12]:
years = list(df[df["Type"] == "advertencia"]['Year'].unique())
years = sorted(years)
years

[2002,
 2003,
 2004,
 2005,
 2006,
 2007,
 2008,
 2009,
 2010,
 2011,
 2012,
 2013,
 2018,
 2019,
 2020,
 2021,
 2022]

# 4. Top 20 Verbs, Adj, and Nouns in Advertencias

In [13]:
for y in years:
    dfg = df[(df["Type"] == "advertencia") & (df["Year"] == y)]
    print(y,"--", len(dfg), "documents")
    dt = printNPOS(' '.join(dfg['Text']), 20)
    display(dt)

2002 -- 27 documents


,Verbs,Abs Freq,Adjectives,Abs Freq,Nouns,Abs Freq
0,hacer,46,armado,165,población,154
1,tener,36,civil,106,riesgo,147
2,recomendar,35,temprano,87,zona,144
3,chocar,35,central,83,municipio,140
4,adoptar,29,alto,56,actor,102
5,garantizar,27,público,50,fuerza,94
6,amenazar,26,forzado,48,medida,94
7,dar,23,social,47,parte,90
8,publicar,21,urbano,45,alerta,89
9,reforzar,20,of,43,comunidad,86


2003 -- 58 documents


,Verbs,Abs Freq,Adjectives,Abs Freq,Nouns,Abs Freq
0,hacer,153,armado,541,población,550
1,tener,107,civil,360,municipio,539
2,dar,106,público,190,riesgo,469
3,encontrar,75,urbano,177,zona,420
4,adoptar,73,alto,174,parte,264
5,garantizar,67,social,174,comunidad,254
6,recomendar,66,selectivo,172,grupo,238
7,realizar,64,ilegal,161,acción,233
8,impedir,62,forzado,152,fuerza,221
9,presentar,60,temprano,141,homicidio,211


2004 -- 88 documents


...

,Verbs,Abs Freq,Adjectives,Abs Freq,Nouns,Abs Freq
0,tener,1521,armado,3075,municipio,4008
1,alertar,1423,temprano,2373,grupo,2726
2,f10,932,vigente,1624,protección,2373
3,hacer,916,ilegal,1594,población,2332
4,encontrar,876,nacional,1476,riesgo,2254
5,presentar,747,código,1315,prevención,2054
6,realizar,625,social,1313,zona,2018
7,permitir,512,territorial,999,territorio,1614
8,generar,508,forzado,932,persona,1538
9,dar,448,rural,929,acción,1481


2021 -- 25 documents


,Verbs,Abs Freq,Adjectives,Abs Freq,Nouns,Abs Freq
0,alertar,724,armado,1402,municipio,2179
1,tener,659,temprano,1320,grupo,1373
2,f10,649,vigente,845,protección,1263
3,encontrar,447,nacional,840,riesgo,1251
4,hacer,403,ilegal,839,población,1189
5,presentar,396,código,708,prevención,1092
6,realizar,372,social,688,zona,1073
7,permitir,279,municipal,571,comunidad,835
8,dar,264,rural,560,acción,830
9,generar,247,territorial,510,territorio,816


2022 -- 14 documents


,Verbs,Abs Freq,Adjectives,Abs Freq,Nouns,Abs Freq
0,tener,567,armado,963,municipio,1143
1,presentar,330,temprano,706,grupo,1010
2,hacer,328,nacional,565,riesgo,928
3,alertar,325,social,548,protección,877
4,f10,276,ilegal,509,prevención,757
5,encontrar,257,alto,446,población,714
6,agc,256,electoral,374,zona,622
7,identificar,208,territorial,346,territorio,578
8,dar,189,vigente,323,parte,569
9,realizar,187,político,322,persona,567


# 5 Top 20 Verbs, Adj, and Nouns in Seguimientos

In [14]:
for y in [2018, 2019]:
    dfg = df[(df["Type"] == "seguimiento") & (df["Year"] == y)]
    print(y,"--", len(dfg), "documents")
    dt = printNPOS(' '.join(dfg['Text']), 20)
    display(dt)

2018 -- 8 documents


,Verbs,Abs Freq,Adjectives,Abs Freq,Nouns,Abs Freq
0,realizar,106,nacional,244,riesgo,206
1,tener,80,armado,148,acción,174
2,hacer,70,alo,112,municipio,166
3,señalar,70,municipal,108,medida,160
4,remitir,70,temprano,100,pueblo,132
5,establecer,64,vigente,98,prevención,124
6,indicar,56,institucional,86,parte,116
7,mitigar,52,público,76,grupo,116
8,encontrar,50,advertido,66,información,108
9,atender,50,efectivo,64,entidad,100


2019 -- 1 documents


,Verbs,Abs Freq,Adjectives,Abs Freq,Nouns,Abs Freq
0,tener,80,social,239,líder,287
1,presentar,32,nacional,75,derechos,236
2,hacer,27,comunitario,72,humanos,234
3,dar,26,público,68,persona,160
4,encontrar,26,comunal,57,defensora,130
5,identificar,26,temprano,51,riesgo,120
6,evidenciar,24,campesino,49,seguimiento,114
7,señalar,20,agrario,45,caso,98
8,chocar,19,armado,43,acción,93
9,permitir,18,indígena,38,protección,90


## 5. Exporting to html

In [15]:
!jupyter nbconvert --to html 4_NLP_Word_Tagging.ipynb

[NbConvertApp] Converting notebook 4_NLP_Word_Tagging.ipynb to html
[NbConvertApp] Writing 666018 bytes to 4_NLP_Word_Tagging.html
